In [2]:
import numpy as np
import pandas as pd
from numpy import savetxt
import matplotlib.pyplot as plt
import MDAnalysis as mda
from numpy.linalg import norm
import MDAnalysis.analysis.atomicdistances as ad
#from MDAnalysis.analysis.dihedrals import Ramachandran
import os

### We are going to calculate the distances between com of ligand 1093 and CA of binding residues.
### We are going to use the com of binding residues for arg from 2 to 11;
### (Not decided yet) we also calculate the distance between com ligand and com of all binding residues' CA as the arg 12 
### Looks good, let's go.

In [3]:
path = './'

topfile = path+"GLYR_6pm5.prmtop"

dcd_path = path + 'DCDs/'
ligc_directory = 'ligC_coordinates/'
ligc_path = path + ligc_directory
preprocessed_directory = 'processed_ligC_coordinates'
distance_matrix_directory = 'distance_matrix'
preprocessed_path = path + preprocessed_directory
distance_matrix_path = path + distance_matrix_directory

pictures_directory = 'pictures'
pictures_path = path + pictures_directory

if os.path.exists(preprocessed_path)==False:
    os.mkdir(preprocessed_path)
    print("Path '%s' is created!"%preprocessed_directory)
else: 
    print("Path '%s' is already existed!"%preprocessed_directory)
    
if os.path.exists(distance_matrix_path)==False:
    os.mkdir(distance_matrix_path)
    print("Path '%s' is created!"%distance_matrix_directory)
else: 
    print("Path '%s' is already existed!"%distance_matrix_directory)    
    
if os.path.exists(ligc_path)==False:
    os.mkdir(ligc_path)
    print("Path '%s' is created!"%ligc_directory)
else: 
    print("Path '%s' is already existed!"%ligc_directory)    
    
if os.path.exists(pictures_path)==False:
    os.mkdir(pictures_path)
    print("Path '%s' is created!"%pictures_directory)
else: 
    print("Path '%s' is already existed!"%pictures_directory) 

Path 'processed_ligC_coordinates' is already existed!
Path 'distance_matrix' is created!
Path 'ligC_coordinates/' is already existed!
Path 'pictures' is already existed!


In [7]:
binding_atoms_index = [13646,13681,13724,12946,12935,11988,7932,8961,8756,7893]
binding_atoms_resid = [u.atoms[i].resid for i in binding_atoms_index]
ligand = 1093

print(binding_atoms_resid)

[850, 852, 855, 807, 806, 747, 495, 559, 547, 493]


In [20]:
### Coordinates of com of ligand 1093
for j in range(1,21):
    u = mda.Universe(topfile,dcd_path+f'mtdFA_{j}.0.dcd')
    lig_coor_lt = []
    for ts in u.trajectory:
        lig_coor = u.select_atoms(f'resid {ligand}').center_of_mass()    
        lig_coor_lt.append(lig_coor)
    np.savetxt(ligc_path+f'ligC_coor{j}.csv',lig_coor_lt,delimiter=',')    

/Users/xueguangpeng/anaconda3/lib/python3.11/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [21]:
### Concatenate all coordinates files together
dfs = []
for j in range(1,21):
    df = pd.read_csv(ligc_path+f'ligC_coor{j}.csv', header=None, delimiter=',')
    dfs.append(df)
concatenated_df = pd.concat(dfs, axis=0, ignore_index=True)
concatenated_df.to_csv(ligc_path+f'ligC_coor_all.csv', index=False)

In [10]:
### Distances between ligand and CA of binding residus
for j in range(1,21):
    u = mda.Universe(topfile,dcd_path+f'mtdFA_{j}.0.dcd')
    for i,arg in enumerate(binding_atoms_resid):
        distance = []
        arg2_index = binding_atoms_resid[i]
        for ts in u.trajectory:
            arg1 = u.select_atoms(f'resid {ligand}').center_of_mass()
            arg2 = u.select_atoms(f'resid {arg2_index} and name CA').positions
            distance.append(norm(arg1 - arg2))
        np.savetxt(distance_matrix_path+f'/dist{i}_traj{j}.csv', distance, delimiter=',')    

In [50]:
### Concatenate all csv files of the same traj into one
for j in range(1,21):
    print(f'Traj {j} is being processed!')
    dfs = []
    for i in range(3,10): ### We don't want to use residues from loop C, which are dist0, dist1, dist2
        # Iterate all sub-csv files from same traj and store them in a list
        df = pd.read_csv(distance_matrix_path+f'/dist{i}_traj{j}.csv', header=None, delimiter=',')
        dfs.append(df)
    # Concatenate all files in the list along the columns (axis=1)
    concatenated_df = pd.concat(dfs, axis=1, ignore_index=True)    
    # Save the concatenated df into file
    concatenated_df.to_csv(distance_matrix_path+f'/dist_all_traj{j}.csv',index=False)

Traj 1 is being processed!
Traj 2 is being processed!
Traj 3 is being processed!
Traj 4 is being processed!
Traj 5 is being processed!
Traj 6 is being processed!
Traj 7 is being processed!
Traj 8 is being processed!
Traj 9 is being processed!
Traj 10 is being processed!
Traj 11 is being processed!
Traj 12 is being processed!
Traj 13 is being processed!
Traj 14 is being processed!
Traj 15 is being processed!
Traj 16 is being processed!
Traj 17 is being processed!
Traj 18 is being processed!
Traj 19 is being processed!
Traj 20 is being processed!


In [51]:
### Concatenate all 'dist_all_traj{j}.csv' files into one
dfs = []
for traj in range(1,21):
    print(f'Traj {traj} is being processed!')
    df = pd.read_csv(distance_matrix_path+f'/dist_all_traj{traj}.csv', delimiter=',')
    dfs.append(df)
concatenated_df = pd.concat(dfs, axis=0, ignore_index=True)
concatenated_df.to_csv(distance_matrix_path+f'/dist_all_traj_all.csv', index=False)

Traj 1 is being processed!
Traj 2 is being processed!
Traj 3 is being processed!
Traj 4 is being processed!
Traj 5 is being processed!
Traj 6 is being processed!
Traj 7 is being processed!
Traj 8 is being processed!
Traj 9 is being processed!
Traj 10 is being processed!
Traj 11 is being processed!
Traj 12 is being processed!
Traj 13 is being processed!
Traj 14 is being processed!
Traj 15 is being processed!
Traj 16 is being processed!
Traj 17 is being processed!
Traj 18 is being processed!
Traj 19 is being processed!
Traj 20 is being processed!


In [52]:
### Concatenate the distance matrix with the ligand coordinates file
df1 = pd.read_csv(ligc_path+str('ligC_coor_all')+'.csv', delimiter=',')
df2 = pd.read_csv(distance_matrix_path+f'/dist_all_traj_all.csv', delimiter=',')
data_som = pd.concat([df1,df2], axis=1, ignore_index=True)
data_som.to_csv(distance_matrix_path+f'/data_som.csv',index=False)

In [53]:
df1

,0,1,2
0,54.128346,35.324667,58.327432
1,54.005430,35.874759,58.957123
2,54.238134,36.024020,58.714942
3,54.211099,35.410845,58.743176
4,54.555177,34.921131,58.336460
...,...,...,...
199090,60.322333,18.073638,63.977756
199091,61.589906,19.921540,63.751187
199092,60.891999,20.187166,64.098845
199093,60.672483,18.529533,64.526087


In [54]:
df2

,0,1,2,3,4,5,6
0,6.123793,8.933876,11.269693,7.736821,6.051318,7.089860,6.851120
1,5.374894,8.251142,11.372989,7.700322,5.886343,6.448066,6.842786
2,5.466425,8.348725,11.602082,7.627951,5.858999,6.894082,6.614444
3,5.783469,8.682888,11.716611,7.640462,6.299142,7.089859,6.512103
4,6.262784,9.276599,12.106043,7.408727,6.082800,7.399085,6.342047
...,...,...,...,...,...,...,...
199090,21.849113,23.777070,28.773979,15.551614,17.732967,19.260982,20.517616
199091,20.622778,22.772599,27.590554,13.563591,15.776109,17.549587,18.600353
199092,19.841974,22.217564,27.253898,13.270518,15.082409,17.111981,18.443255
199093,21.338511,23.620795,28.583325,14.809887,17.030677,18.637680,19.774320


In [55]:
data = pd.read_csv(distance_matrix_path+f'/data_som.csv',header=0,delimiter=',')
data

,0,1,2,3,4,5,6,7,8,9
0,54.128346,35.324667,58.327432,6.123793,8.933876,11.269693,7.736821,6.051318,7.089860,6.851120
1,54.005430,35.874759,58.957123,5.374894,8.251142,11.372989,7.700322,5.886343,6.448066,6.842786
2,54.238134,36.024020,58.714942,5.466425,8.348725,11.602082,7.627951,5.858999,6.894082,6.614444
3,54.211099,35.410845,58.743176,5.783469,8.682888,11.716611,7.640462,6.299142,7.089859,6.512103
4,54.555177,34.921131,58.336460,6.262784,9.276599,12.106043,7.408727,6.082800,7.399085,6.342047
...,...,...,...,...,...,...,...,...,...,...
199090,60.322333,18.073638,63.977756,21.849113,23.777070,28.773979,15.551614,17.732967,19.260982,20.517616
199091,61.589906,19.921540,63.751187,20.622778,22.772599,27.590554,13.563591,15.776109,17.549587,18.600353
199092,60.891999,20.187166,64.098845,19.841974,22.217564,27.253898,13.270518,15.082409,17.111981,18.443255
199093,60.672483,18.529533,64.526087,21.338511,23.620795,28.583325,14.809887,17.030677,18.637680,19.774320


In [56]:
data.shape

(199095, 10)

In [57]:
data = np.loadtxt(distance_matrix_path+f'/data_som.csv',skiprows=1,delimiter=',')

In [48]:
data

array([[54.12834611, 35.32466713, 58.32743231, ...,  6.05131771,
         7.08986009,  6.85111978],
       [54.00543031, 35.87475856, 58.95712263, ...,  5.88634336,
         6.44806602,  6.84278552],
       [54.23813412, 36.02401961, 58.71494165, ...,  5.8589987 ,
         6.89408159,  6.61444371],
       ...,
       [60.89199948, 20.18716626, 64.09884503, ..., 15.08240896,
        17.11198077, 18.44325537],
       [60.6724828 , 18.52953263, 64.52608688, ..., 17.03067665,
        18.63767961, 19.77431953],
       [62.28208033, 16.77533363, 62.92820575, ..., 18.70192156,
        20.92774939, 21.2545896 ]])

In [49]:
data.shape

(199095, 13)